In [ ]:
import pyspark.sql.functions as fn

In [ ]:
import pyspark.sql.types as t

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pickle

# Load from Parquet

In [ ]:
df = spark.read.parquet('../dataset/M20_cards_features.parquet')

In [ ]:
df.createOrReplaceTempView("cards_features")

In [ ]:
df.columns

# Groups 

## By text features 

In [ ]:
lenc = None

with open("../models/pyspark/labelencoder_text_feats", "rb") as fp:
    lenc = pickle.load(fp)

In [ ]:
if 'text_featuers_enc' in df.columns:
    df = df.drop("text_featuers_enc")

df = df.withColumn("text_features_enc", fn.explode(df.text_features_vect))

In [ ]:
df_count = df.select(["text_features_enc", "colorIdentity"]).groupBy('text_features_enc').count().orderBy('text_features_enc', ascending=True)

In [ ]:
df_count.toPandas()

In [ ]:
@fn.udf#(returnType=t.StringType())
def resolve_encoded_label(label):
    label = int(label)

    result = lenc.inverse_transform([label])[0]
    result = str(result)
    
    return result

In [ ]:
if "name" in df_count.columns:
    df_count.drop("name")

df_count = df_count.withColumn('name', resolve_encoded_label('text_features_enc'))

In [ ]:
pd_count = df_count.toPandas()

In [ ]:
pd_count

In [ ]:
sns.barplot(x="count", y="name", data=pd_count, orient="h")

In [ ]:
df_count_mult = df.select(["text_features_enc", "colorIdentity"]).\
                    groupBy(['text_features_enc', "colorIdentity"]).count().\
                    orderBy('text_features_enc', ascending=True)

In [ ]:
df_count_mult = df_count_mult.withColumn('name', resolve_encoded_label('text_features_enc'))

In [ ]:
pd_count_mult = df_count_mult.toPandas()

In [ ]:
pd_count_mult

## By color identity and text features

In [ ]:
pivot = pd_count_mult.pivot("name", "colorIdentity", "count")

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, figsize=(10, 10), dpi=100, sharex=True, sharey=False, gridspec_kw={"height_ratios": (.10, .90)})
 
g = sns.lineplot(x="colorIdentity", y="count", data=pd_count_mult, ci=None, ax=ax_box)
# g = sns.countplot(x="colorIdentity", data=pd_count_mult, ax=ax_box)
sns.despine(top=True, bottom=True, left=True, right=True, offset=0.0)
ax_box.set(yticks=[])
ax_box.set(xticks=[])
ax_box.set(ylabel="")
ax_box.set(xlabel="")
ax_box.set(xlim=(-0.5, 23))
ax_box.set_xticklabels(range(0,23))

sns.heatmap(pivot, square=True, cmap="YlGnBu", cbar=False, linewidths=1.0, annot=True, ax=ax_hist)
